In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
class emt_peru_crawler():
    def __init__(self, base_url):
        if type(base_url) != str:
            raise("base_url must be str type.")
        if "elmejortrato.com.pe" not in base_url:
            raise("base_url must contain elmejortrato.com.pe")
        self._ratings = []
        self._base_url = base_url
    
    def crawl(self):
        '''
        Inicia todo el proceso.
        Cuando se inicializa la clase, se debe inicializar con una base_url desde la cual
        se empezará el proceso. Esta url_base tiene que contener un listado de posts padres.
        '''
        r = requests.get(self._base_url) 
        soup = BeautifulSoup(r.content, 'html5lib') 
        topics_container = soup.find('section', attrs={'class':'results'})
        for topic in topics_container.findAll('a'):
            print("Crawling parent post "+str(topic['href']))
            parent_post_soup = self.get_soup(topic['href'])
            self.get_post_ratings(parent_post_soup)
            self.get_children_posts_ratings(parent_post_soup)
            
    def get_soup(self, url):
        '''
        Devuelve el contenido html de una url específica.
        '''
        if "https://elmejortrato.com.pe" not in url:
            url = "https://elmejortrato.com.pe" + url
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html5lib')
        return soup
    
    def get_post_ratings(self, post_soup):
        '''
        Recibe el html de un post y busca todos los ratings que haya en ese post.
        Retorna un array con el contenido de todos los ratings.
        '''
        ratings_container = post_soup.find('section', attrs = {'id':'califications'}) 
        ratings = []
        for row in ratings_container.findAll('div', attrs = {'class':'cal'}): 
            rating = {
                'text': row.p.text,
                'state': 1 # 1=Approved
            } 
            self._ratings.append(rating)
    
    def get_children_posts_ratings(self, parent_post_soup):
        '''
        Cada post, tiene al fondo una sección de articulos relacionados o "hijos".
        En este método, se busca esa sección y se va a cada uno de los posts que estan en esa
        sección a buscar sus ratings.
        '''
        related_section = parent_post_soup.find('div',attrs={'id':'relations'})
        if related_section is not None:
            children_url = ["https://elmejortrato.com.pe" + url['href'] for url in related_section.findAll('a')]
            for url in children_url:
                print("Crawling child post "+str(url))
                child_post_soup = self.get_soup(url)
                self.get_post_ratings(child_post_soup)
            
    def export_to_csv(self, filename):
        if len(self._ratings) > 0:
            df = pd.DataFrame(data=self._ratings)
            df.to_csv(filename, index=False,encoding='utf-8')
        else:
            raise("Ratings size must be greater than 0.")

In [3]:
%%time
seguros_por_aseguradora = emt_peru_crawler("https://www.elmejortrato.com.pe/seguros/seguros-por-aseguradora")
seguros_por_aseguradora.crawl()
seguros_por_aseguradora.export_to_csv('./data/seguros-por-aseguradora.csv')

Crawling parent post /seguros/hdi-seguros-vehiculares-en-peru
Crawling child post https://elmejortrato.com.pe/seguros/hdi-seguros-vehiculares-en-peru/direccion-y-telefono-de-oficinas-de-hdi-seguros-en-peru
Crawling parent post /seguros/seguros-vehiculares-mapfre
Crawling child post https://elmejortrato.com.pe/seguros/seguros-vehiculares-mapfre/seguros-mapfre-para-autos-en-cusco
Crawling child post https://elmejortrato.com.pe/seguros/seguros-vehiculares-mapfre/seguros-mapfre-en-miraflores
Crawling child post https://elmejortrato.com.pe/seguros/seguros-vehiculares-mapfre/seguros-mapfre-en-chiclayo
Crawling child post https://elmejortrato.com.pe/seguros/seguros-vehiculares-mapfre/seguros-mapfre-en-tacna
Crawling child post https://elmejortrato.com.pe/seguros/seguros-vehiculares-mapfre/seguros-mapfre-en-trujillo-peru
Crawling child post https://elmejortrato.com.pe/seguros/seguros-vehiculares-mapfre/telefonos-de-seguros-mapfre
Crawling child post https://elmejortrato.com.pe/seguros/seguros-

In [4]:
%%time
seguros_por_provincia = emt_peru_crawler("https://www.elmejortrato.com.pe/seguros/seguros-por-provincia")
seguros_por_provincia.crawl()
seguros_por_provincia.export_to_csv('./data/seguros-por-provincia.csv')

Crawling parent post /seguros/seguro-de-auto-en-moquegua
Crawling parent post /seguros/seguro-de-auto-en-arequipa
Crawling parent post /seguros/seguro-de-auto-en-huaral
Crawling parent post /seguros/seguro-vehicular-en-puerto-maldonado
Crawling parent post /seguros/seguro-para-auto-en-cusco
Crawling parent post /seguros/seguro-vehicular-en-huancayo
Crawling parent post /seguros/seguro-para-auto-en-huanuco
Crawling parent post /seguros/seguro-para-auto-en-lima
Crawling parent post /seguros/seguro-vehicular-en-tingo-maria
Crawling parent post /seguros/seguro-de-auto-en-paita
Crawling parent post /seguros/seguro-vehicular-en-callao
Crawling parent post /seguros/seguro-para-auto-en-san-martin
Crawling parent post /seguros/seguro-vehicular-en-juliaca
Crawling parent post /seguros/seguro-de-auto-en-tacna
Crawling parent post /seguros/seguro-para-auto-en-cajamarca
Crawling parent post /seguros/seguro-de-auto-en-jaen
Crawling parent post /seguros/seguro-de-auto-en-chimbote
Crawling parent post

In [5]:
%%time
seguros_por_auto = emt_peru_crawler("https://www.elmejortrato.com.pe/seguros/seguros-por-auto")
seguros_por_auto.crawl()
seguros_por_auto.export_to_csv('./data/seguros-por-auto.csv')

Crawling parent post /seguros/donde-contratar-seguro-vehicular-para-toyota
Crawling child post https://elmejortrato.com.pe/seguros/donde-contratar-seguro-vehicular-para-toyota/como-contratar-un-seguro-vehicular-para-toyota-corolla
Crawling child post https://elmejortrato.com.pe/seguros/donde-contratar-seguro-vehicular-para-toyota/que-seguro-de-auto-conviene-para-toyota-hilux
Crawling child post https://elmejortrato.com.pe/seguros/donde-contratar-seguro-vehicular-para-toyota/donde-contratar-seguro-vehicular-para-toyota-yaris
Crawling parent post /seguros/como-contratar-un-seguro-vehicular-para-volkswagen
Crawling parent post /seguros/que-seguro-de-auto-conviene-para-renault
Crawling parent post /seguros/que-seguro-de-auto-conviene-para-kia
Crawling child post https://elmejortrato.com.pe/seguros/que-seguro-de-auto-conviene-para-kia/seguros-vehiculares-para-kia-picanto
Crawling child post https://elmejortrato.com.pe/seguros/que-seguro-de-auto-conviene-para-kia/seguros-para-autos-chevrolet

In [6]:
%%time
seguros_por_tipo = emt_peru_crawler("https://www.elmejortrato.com.pe/seguros/seguros-por-tipo")
seguros_por_tipo.crawl()
seguros_por_tipo.export_to_csv('./data/seguros-por-tipo.csv')

Crawling parent post /seguros/seguro-vehicular-soat-peru
Crawling child post https://elmejortrato.com.pe/seguros/seguro-vehicular-soat-peru/seguro-vehicular-afocat
Crawling child post https://elmejortrato.com.pe/seguros/seguro-vehicular-soat-peru/soat-para-minivan-particular
Crawling parent post /seguros/impuesto-vehicular-peru
Crawling parent post /seguros/seguro-para-vehiculos
Crawling child post https://elmejortrato.com.pe/seguros/seguro-para-vehiculos/seguro-vehicular-para-camiones
Crawling child post https://elmejortrato.com.pe/seguros/seguro-para-vehiculos/que-cubre-el-seguro-vehicular
Crawling child post https://elmejortrato.com.pe/seguros/seguro-para-vehiculos/seguro-simple-auto
Crawling child post https://elmejortrato.com.pe/seguros/seguro-para-vehiculos/seguros-vehiculos-transporte-pasajeros
Wall time: 8.79 s


In [6]:
por_aseguradora_df = pd.read_csv('./data/seguros-por-aseguradora.csv')
por_auto_df = pd.read_csv('./data/seguros-por-auto.csv')
por_provincia_df = pd.read_csv('./data/seguros-por-provincia.csv')
por_tipo_df = pd.read_csv('./data/seguros-por-tipo.csv')
aprobados = pd.read_csv('./data/aprobados.csv')
approved_df = pd.concat([por_aseguradora_df,por_auto_df,por_provincia_df,por_tipo_df, aprobados])
approved_df.to_csv("./data/all_approved.csv", index=False)